# Manual “USC” experiments

See [`usc.ipynb`](usc.ipynb) for the main “USC” notebook.

In [1]:
from lxml import etree as ET
import openai
import tiktoken

import embed

## Small semantic search experiment with fragments

In [2]:
# § 112b - United States international agreements; transmission to Congress
S112B = """<section style="-uslm-lc:I80" id="idc8f8d033-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b"><num value="112b">§ 112b.</num><heading> United States international agreements; transmission to Congress</heading><subsection style="-uslm-lc:I11" class="indent0" id="idc8f8d034-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/a"><num value="a">(a)</num><content> The Secretary of State shall transmit to the Congress the text of any international agreement (including the text of any oral international agreement, which agreement shall be reduced to writing), other than a treaty, to which the United States is a party as soon as practicable after such agreement has entered into force with respect to the United States but in no event later than sixty days thereafter. However, any such agreement the immediate public disclosure of which would, in the opinion of the President, be prejudicial to the national security of the United States shall not be so transmitted to the Congress but shall be transmitted to the Committee on Foreign Relations of the Senate and the Committee on International Relations of the House of Representatives under an appropriate injunction of secrecy to be removed only upon due notice from the President. Any department or agency of the United States Government which enters into any international agreement on behalf of the United States shall transmit to the Department of State the text of such agreement not later than twenty days after such agreement has been signed.</content>
</subsection>
<subsection style="-uslm-lc:I11" class="indent0" id="idc8fb1a25-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/b"><num value="b">(b)</num><content> Not later than <date date="1979-03-01">March 1, 1979</date>, and at yearly intervals thereafter, the President shall, under his own signature, transmit to the Speaker of the House of Representatives and the chairman of the Committee on Foreign Relations of the Senate a report with respect to each international agreement which, during the preceding year, was transmitted to the Congress after the expiration of the 60-day period referred to in the first sentence of subsection (a), describing fully and completely the reasons for the late transmittal.</content>
</subsection>
<subsection style="-uslm-lc:I11" class="indent0" id="idc8fb1a26-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/c"><num value="c">(c)</num><content> Notwithstanding any other provision of law, an international agreement may not be signed or otherwise concluded on behalf of the United States without prior consultation with the Secretary of State. Such consultation may encompass a class of agreements rather than a particular agreement.</content>
</subsection>
<subsection style="-uslm-lc:I11" class="indent0" id="idc8fb1a27-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/d"><num value="d">(d)</num><paragraph style="-uslm-lc:I11" class="indent0" id="idc8fb1a28-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/d/1"><num value="1">(1)</num><chapeau> The Secretary of State shall annually submit to Congress a report that contains an index of all international agreements, listed by country, date, title, and summary of each such agreement (including a description of the duration of activities under the agreement and the agreement itself), that the United States—</chapeau><subparagraph style="-uslm-lc:I12" class="indent1" id="idc8fb1a29-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/d/1/A"><num value="A">(A)</num><content> has signed, proclaimed, or with reference to which any other final formality has been executed, or that has been extended or otherwise modified, during the preceding calendar year; and</content>
</subparagraph>
<subparagraph style="-uslm-lc:I12" class="indent1" id="idc8fb1a2a-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/d/1/B"><num value="B">(B)</num><content> has not been published, or is not proposed to be published, in the compilation entitled “United States Treaties and Other International Agreements”.</content>
</subparagraph>
</paragraph>
<paragraph style="-uslm-lc:I11" class="indent0" id="idc8fb1a2b-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/d/2"><num value="2">(2)</num><content> The report described in paragraph (1) may be submitted in classified form.</content>
</paragraph>
</subsection>
<subsection style="-uslm-lc:I11" class="indent0" id="idc8fb1a2c-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/e"><num value="e">(e)</num><paragraph style="-uslm-lc:I11" class="indent0" id="idc8fb1a2d-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/e/1"><num value="1">(1)</num><content> Subject to paragraph (2), the Secretary of State shall determine for and within the executive branch whether an arrangement constitutes an international agreement within the meaning of this section.</content>
</paragraph>
<paragraph style="-uslm-lc:I11" class="indent0" id="idc8fb1a2e-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/e/2"><num value="2">(2)</num><subparagraph style="-uslm-lc:I11" class="indent0" id="idc8fb1a2f-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/e/2/A"><num value="A">(A)</num><content> An arrangement shall constitute an international agreement within the meaning of this section (other than subsection (c)) irrespective of the duration of activities under the arrangement or the arrangement itself.</content>
</subparagraph>
<subparagraph style="-uslm-lc:I11" class="indent0" id="idc8fb1a30-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/e/2/B"><num value="B">(B)</num><chapeau> Arrangements that constitute an international agreement within the meaning of this section (other than subsection (c)) include the following:</chapeau><clause style="-uslm-lc:I12" class="indent1" id="idc8fb1a31-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/e/2/B/i"><num value="i">(i)</num><content> A bilateral or multilateral counterterrorism agreement.</content>
</clause>
<clause style="-uslm-lc:I12" class="indent1" id="idc8fb1a32-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/e/2/B/ii"><num value="ii">(ii)</num><content> A bilateral agreement with a country that is subject to a determination under section 6(j)(1)(A) <ref class="footnoteRef" idref="fn002006">1</ref><note type="footnote" id="fn002006"><num>1</num> See References in Text note below.</note> of the Export Administration Act of 1979 (50 U.S.C. App. 2405(j)(1)(A)), section 620A(a) of the Foreign Assistance Act of 1961 (<ref href="/us/usc/t22/s2371/a">22 U.S.C. 2371(a)</ref>), or section 40(d) of the Arms Export Control Act (<ref href="/us/usc/t22/s2780/d">22 U.S.C. 2780(d)</ref>).</content>
</clause>
</subparagraph>
</paragraph>
</subsection>
<subsection style="-uslm-lc:I11" class="indent0" id="idc8fb1a33-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/f"><num value="f">(f)</num><content> The President shall, through the Secretary of State, promulgate such rules and regulations as may be necessary to carry out this section.</content>
</subsection>
<subsection style="-uslm-lc:I11" class="indent0" id="idc8fb1a34-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/g"><num value="g">(g)</num><content> It is the sense of Congress that the executive branch should not prescribe or otherwise commit to or include specific legislative text in a treaty or executive agreement unless Congress has authorized such action.</content>
</subsection>
<sourceCredit id="idc8fb1a35-d96f-11ed-bd9a-904c91f2bbbe">(Added <ref href="/us/pl/92/403/s1">Pub. L. 92–403, § 1</ref>, <date date="1972-08-22">Aug. 22, 1972</date>, <ref href="/us/stat/86/619">86 Stat. 619</ref>; amended <ref href="/us/pl/95/45/s5">Pub. L. 95–45, § 5</ref>, <date date="1977-06-15">June 15, 1977</date>, <ref href="/us/stat/91/224">91 Stat. 224</ref>; <ref href="/us/pl/95/426/tVII/s708">Pub. L. 95–426, title VII, § 708</ref>, <date date="1978-10-07">Oct. 7, 1978</date>, <ref href="/us/stat/92/993">92 Stat. 993</ref>; <ref href="/us/pl/103/437/s1">Pub. L. 103–437, § 1</ref>, <date date="1994-11-02">Nov. 2, 1994</date>, <ref href="/us/stat/108/4581">108 Stat. 4581</ref>; <ref href="/us/pl/108/458/tVII/s7121/b">Pub. L. 108–458, title VII, § 7121(b)</ref>–(d), <date date="2004-12-17">Dec. 17, 2004</date>, <ref href="/us/stat/118/3807">118 Stat. 3807</ref>, 3808; <ref href="/us/pl/116/260/dFF/tXVII/s1708/b">Pub. L. 116–260, div. FF, title XVII, § 1708(b)</ref>, <date date="2020-12-27">Dec. 27, 2020</date>, <ref href="/us/stat/134/3298">134 Stat. 3298</ref>; <ref href="/us/pl/117/263/dE/tLIX/s5947/a/1">Pub. L. 117–263, div. E, title LIX, § 5947(a)(1)</ref>, <date date="2022-12-23">Dec. 23, 2022</date>, <ref href="/us/stat/136/3476">136 Stat. 3476</ref>.)</sourceCredit>
<notes type="uscNote" id="idc8fb1a36-d96f-11ed-bd9a-904c91f2bbbe">
<note style="-uslm-lc:I84" topic="prospectiveAmendment" id="idc8fb1a37-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered fontsize8 smallCaps">Amendment of Section</heading><p><ref href="/us/pl/117/263/dE/tLIX/s5947/a/1">Pub. L. 117–263, div. E, title LIX, § 5947(a)(1)</ref>, (c), <date date="2022-12-23">Dec. 23, 2022</date>, <ref href="/us/stat/136/3476">136 Stat. 3476</ref>, 3482, provided that, effective 270 days after <date date="2022-12-22">Dec. 22, 2022</date>, this section is amended to read as follows:<quotedContent>
<p style="-uslm-lc:I68" class="indent2 firstIndent-2 fontsize8 italic">§ 112b. United States international agreements and non-binding instruments; transparency provisions</p>
</quotedContent>
</p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(a)(1) Not less frequently than once each month, the Secretary shall provide in writing to the Majority Leader of the Senate, the Minority Leader of the Senate, the Speaker of the House of Representatives, the Minority Leader of the House of Representatives, and the appropriate congressional committees the following:</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(A)(i) A list of all international agreements and qualifying non-binding instruments signed, concluded, or otherwise finalized during the prior month.</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(ii) The text of all international agreements and qualifying non-binding instruments described in clause (i).</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(iii) A detailed description of the legal authority that, in the view of the Secretary, provides authorization for each international agreement and that, in the view of the appropriate department or agency, provides authorization for each qualifying non-binding instrument provided under clause (ii) to become operative. If multiple authorities are relied upon in relation to an international agreement, the Secretary shall cite all such authorities, and if multiple authorities are relied upon in relation to a qualifying non-binding instrument, the appropriate department or agency shall cite all such authorities. All citations to the Constitution of the United States, a treaty, or a statute shall include the specific article or section and subsection reference whenever available and, if not available, shall be as specific as possible. If the authority relied upon is or includes article II of the Constitution of the United States, the Secretary or appropriate department or agency shall explain the basis for that reliance.</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(B)(i) A list of all international agreements that entered into force and qualifying non-binding instruments that became operative for the United States or an agency of the United States during the prior month.</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(ii) The text of all international agreements and qualifying non-binding instruments described in clause (i) if such text differs from the text of the agreement or instrument previously provided pursuant to subparagraph (A)(ii).</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(iii) A statement describing any new or amended statutory or regulatory authority anticipated to be required to fully implement each proposed international agreement and qualifying non-binding instrument included in the list described in clause (i).</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(2) The information and text required by paragraph (1) shall be submitted in unclassified form, but may include a classified annex.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(b)(1) Not later than 120 days after the date on which an international agreement enters into force, the Secretary shall make the text of the agreement, and the information described in subparagraphs (A)(iii) and (B)(iii) of subsection (a)(1) relating to the agreement, available to the public on the website of the Department of State.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(2) Not less frequently than once every 120 days, the Secretary shall make the text of each qualifying non-binding instrument that became operative during the preceding 120 days, and the information described in subparagraphs (A)(iii) and (B)(iii) of subsection (a)(1) relating to each such instrument, available to the public on the website of the Department of State.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(3) The requirements under paragraphs (1) and (2) shall not apply to the following categories of international agreements or qualifying non-binding instruments, or to information described in subparagraphs (A)(iii) and (B)(iii) of subsection (a)(1) relating to such agreements or qualifying non-binding instruments:</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(A) International agreements and qualifying non-binding instruments that contain information that has been given a national security classification pursuant to Executive Order 13526 (<ref href="/us/usc/t50/s3161">50 U.S.C. 3161</ref> note; relating to classified national security information) or any predecessor or successor order, or that contain any information that is otherwise exempt from public disclosure pursuant to United States law.</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(B) International agreements and qualifying non-binding instruments that address military operations, military exercises, acquisition and cross servicing, logistics support, military personnel exchange or education programs, or the provision of health care to military personnel on a reciprocal basis.</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(C) International agreements and qualifying non-binding instruments that establish the terms of grant or other similar assistance, including in-kind assistance, financed with foreign assistance funds pursuant to the Foreign Assistance Act of 1961 (<ref href="/us/usc/t22/s2151">22 U.S.C. 2151</ref> et seq.) or the Food for Peace Act (<ref href="/us/usc/t7/s1691">7 U.S.C. 1691</ref> et seq.).</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(D) International agreements and qualifying non-binding instruments, such as project annexes and other similar instruments, for which the principal function is to establish technical details for the implementation of a specific project undertaken pursuant to another agreement or qualifying non-binding instrument that has been published in accordance with paragraph (1) or (2).</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(E) International agreements and qualifying non-binding instruments that have been separately published by a depositary or other similar administrative body, except that the Secretary shall make the information described in subparagraphs (A)(iii) and (B)(iii) of subsection (a)(1), relating to such agreements or qualifying non-binding instruments, available to the public on the website of the Department of State within the timeframes required by paragraph (1) or (2).</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(c) For any international agreement or qualifying non-binding instrument for which an implementing agreement or arrangement, or any document of similar purpose or function to the aforementioned regardless of the title of the document, is not otherwise required to be submitted to the Majority Leader of the Senate, the Minority Leader of the Senate, the Speaker of the House of Representatives, the Minority Leader of the House of Representatives, and the appropriate congressional committees under subparagraphs (A)(ii) or (B)(ii) of subsection (a)(1), not later than 30 days after the date on which the Secretary receives a written communication from the Chair or Ranking Member of either of the appropriate congressional committees requesting the text of any such implementing agreements or arrangements, whether binding or non-binding, the Secretary shall submit such implementing agreements or arrangements to the Majority Leader of the Senate, the Minority Leader of the Senate, the Speaker of the House of Representatives, the Minority Leader of the House of Representatives, and the appropriate congressional committees.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(d) Any department or agency of the United States Government that enters into any international agreement or qualifying non-binding instrument on behalf of itself or the United States shall—</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(1) provide to the Secretary the text of each international agreement not later than 15 days after the date on which such agreement is signed or otherwise concluded;</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(2) provide to the Secretary the text of each qualifying non-binding instrument not later than 15 days after the date on which such instrument is concluded or otherwise becomes finalized;</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(3) provide to the Secretary a detailed description of the legal authority that provides authorization for each qualifying non-binding instrument to become operative not later than 15 days after such instrument is signed or otherwise becomes finalized; and</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(4) on an ongoing basis, provide any implementing material to the Secretary for transmittal to the Majority Leader of the Senate, the Minority Leader of the Senate, the Speaker of the House of Representatives, the Minority Leader of the House of Representatives, and the appropriate congressional committees as needed to satisfy the requirements described in subsection (c).</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(e)(1) Each department or agency of the United States Government that enters into any international agreement or qualifying non-binding instrument on behalf of itself or the United States shall designate a Chief International Agreements Officer, who shall—</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(A) be selected from among employees of such department or agency;</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(B) serve concurrently as the Chief International Agreements Officer; and</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(C) subject to the authority of the head of such department or agency, have department- or agency-wide responsibility for efficient and appropriate compliance with this section.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(2) There shall be a Chief International Agreements Officer who serves at the Department of State with the title of International Agreements Compliance Officer.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(f) The substance of oral international agreements shall be reduced to writing for the purpose of meeting the requirements of subsections (a) and (b).</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(g) Notwithstanding any other provision of law, an international agreement may not be signed or otherwise concluded on behalf of the United States without prior consultation with the Secretary. Such consultation may encompass a class of agreements rather than a particular agreement.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(h)(1) Not later than 3 years after the date of the enactment of this section, and not less frequently than once every 3 years thereafter during the 9-year period beginning on the date of the enactment of this section, the Comptroller General of the United States shall conduct an audit of the compliance of the Secretary with the requirements of this section.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(2) In any instance in which a failure by the Secretary to comply with such requirements is determined by the Comptroller General to have been due to the failure or refusal of another agency to provide information or material to the Department of State, or the failure to do so in a timely manner, the Comptroller General shall engage such other agency to determine—</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(A) the cause and scope of such failure or refusal;</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(B) the specific office or offices responsible for such failure or refusal; and</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(C) recommendations for measures to ensure compliance with statutory requirements.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(3) The Comptroller General shall submit to the Majority Leader of the Senate, the Minority Leader of the Senate, the Speaker of the House of Representatives, the Minority Leader of the House of Representatives, and the appropriate congressional committees in writing the results of each audit required by paragraph (1).</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(4) The Comptroller General and the Secretary shall make the results of each audit required by paragraph (1) publicly available on the websites of the Government Accountability Office and the Department of State, respectively.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(i) The President shall, through the Secretary, promulgate such rules and regulations as may be necessary to carry out this section.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(j) It is the sense of Congress that the executive branch should not prescribe or otherwise commit to or include specific legislative text in a treaty, executive agreement, or non-binding instrument unless Congress has authorized such action.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(k) In this section:</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(1) The term “appropriate congressional committees” means—</i></p>
<p style="-uslm-lc:I13" class="indent2 fontsize8"><i>(A) the Committee on Foreign Relations of the Senate; and</i></p>
<p style="-uslm-lc:I13" class="indent2 fontsize8"><i>(B) the Committee on Foreign Affairs of the House of Representatives.</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(2) The term “appropriate department or agency” means the department or agency of the United States Government that negotiates and enters into a qualifying non-binding instrument on behalf of itself or the United States.</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(3) The term “intelligence community” has the meaning given that term in section 3(4) of the National Security Act of 1947 (<ref href="/us/usc/t50/s3003/4">50 U.S.C. 3003(4)</ref>).</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(4) The term “international agreement” includes—</i></p>
<p style="-uslm-lc:I13" class="indent2 fontsize8"><i>(A) any treaty that requires the advice and consent of the Senate, pursuant to article II of the Constitution of the United States; and</i></p>
<p style="-uslm-lc:I13" class="indent2 fontsize8"><i>(B) any other international agreement to which the United States is a party and that is not subject to the advice and consent of the Senate.</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(5) The term “qualifying non-binding instrument”—</i></p>
<p style="-uslm-lc:I13" class="indent2 fontsize8"><i>(A) except as provided in subparagraph (B), means a non-binding instrument that—</i></p>
<p style="-uslm-lc:I14" class="indent3 fontsize8"><i>(i) is or will be under negotiation, is signed or otherwise becomes operative, or is implemented with one or more foreign governments, international organizations, or foreign entities, including non-state actors; and</i></p>
<p style="-uslm-lc:I14" class="indent3 fontsize8"><i>(ii)(I) could reasonably be expected to have a significant impact on the foreign policy of the United States; or</i></p>
<p style="-uslm-lc:I14" class="indent3 fontsize8"><i>(II) is the subject of a written communication from the Chair or Ranking Member of either of the appropriate congressional committees to the Secretary; and</i></p>
<p style="-uslm-lc:I13" class="indent2 fontsize8">(B) does not include any non-binding instrument that is signed or otherwise becomes operative or is implemented pursuant to the authorities relied upon by the Department of Defense, the Armed Forces of the United States, or any element of the intelligence community.</p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(6) The term “Secretary” means the Secretary of State.</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(7)(A) The term “text” with respect to an international agreement or qualifying non-binding instrument includes—</i></p>
<p style="-uslm-lc:I13" class="indent2 fontsize8"><i>(i) any annex, appendix, codicil, side agreement, side letter, or any document of similar purpose or function to the aforementioned, regardless of the title of the document, that is entered into contemporaneously and in conjunction with the international agreement or qualifying non-binding instrument; and</i></p>
<p style="-uslm-lc:I13" class="indent2 fontsize8"><i>(ii) any implementing agreement or arrangement, or any document of similar purpose or function to the aforementioned regardless of the title of the document, that is entered into contemporaneously and in conjunction with the international agreement or qualifying non-binding instrument.</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(B) As used in subparagraph (A), the term “contemporaneously and in conjunction with”—</i></p>
<p style="-uslm-lc:I13" class="indent2 fontsize8"><i>(i) shall be construed liberally; and</i></p>
<p style="-uslm-lc:I13" class="indent2 fontsize8"><i>(ii) may not be interpreted to require any action to have occurred simultaneously or on the same day.</i></p>
<p style="-uslm-lc:I11" class="indent0 fontsize8"><i>(l) Nothing in this section may be construed—</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(1) to authorize the withholding from disclosure to the public of any record if such disclosure is required by law; or</i></p>
<p style="-uslm-lc:I12" class="indent1 fontsize8"><i>(2) to require the provision of any implementing agreement or arrangement, or any document of similar purpose or function regardless of its title, which was entered into by the Department of Defense, the Armed Forces of the United States, or any element of the intelligence community or any implementing material originating with the aforementioned agencies, if such implementing agreement, arrangement, document, or material was not required to be provided to the Majority Leader of the Senate, the Minority Leader of the Senate, the Speaker of the House of Representatives, the Minority Leader of the House of Representatives, or the appropriate congressional committees prior to the date of the enactment of the James M. Inhofe National Defense Authorization Act for Fiscal Year 2023.</i></p>
<p style="-uslm-lc:I88" class="indent1 fontsize8 italic">See 2022 Amendment note below.</p>
</note>
<note style="-uslm-lc:I74" role="crossHeading" topic="editorialNotes" id="idc8fb1a38-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered"><b>Editorial Notes</b></heading></note>
<note style="-uslm-lc:I75" topic="referencesInText" id="idc8fb1a39-d96f-11ed-bd9a-904c91f2bbbe">

<heading class="centered smallCaps">References in Text</heading><p style="-uslm-lc:I21" class="indent0">Section 6(j)(1)(A) of the Export Administration Act of 1979, referred to in subsec. (e)(2)(B)(ii), which was classified to section 2405(j)(1)(A) of the former Appendix to Title 50, War and National Defense, prior to editorial reclassification and renumbering as <ref href="/us/usc/t50/s4605/j/1/A">section 4605(j)(1)(A) of Title 50</ref>, was repealed by <ref href="/us/pl/115/232/dA/tXVII/s1766/a">Pub. L. 115–232, div. A, title XVII, § 1766(a)</ref>, <date date="2018-08-13">Aug. 13, 2018</date>, <ref href="/us/stat/132/2232">132 Stat. 2232</ref>.</p>
</note>
<note style="-uslm-lc:I74" topic="amendments" id="idc8fb1a3a-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Amendments</heading><p style="-uslm-lc:I21" class="indent0">2022—<ref href="/us/pl/117/263">Pub. L. 117–263</ref> amended section generally. Prior to amendment, section related to requirements for transmitting the text of United States international agreements and various reports to Congress.</p>
<p style="-uslm-lc:I21" class="indent0">2020—Subsec. (g). <ref href="/us/pl/116/260">Pub. L. 116–260</ref> added subsec. (g).</p>
<p style="-uslm-lc:I21" class="indent0">2004—Subsec. (a). <ref href="/us/pl/108/458/s7121/b">Pub. L. 108–458, § 7121(b)</ref>, substituted “Committee on International Relations” for “Committee on Foreign Affairs”.</p>
<p style="-uslm-lc:I21" class="indent0">Subsec. (d). <ref href="/us/pl/108/458/s7121/c">Pub. L. 108–458, § 7121(c)</ref>, added subsec. (d). Former subsec. (d) redesignated (e).</p>
<p style="-uslm-lc:I21" class="indent0">Subsec. (e). <ref href="/us/pl/108/458/s7121/d">Pub. L. 108–458, § 7121(d)</ref>, designated existing provisions as par. (1), substituted “Subject to paragraph (2), the Secretary of State” for “The Secretary of State”, and added par. (2).</p>
<p style="-uslm-lc:I21" class="indent0"><ref href="/us/pl/108/458/s7121/c/1">Pub. L. 108–458, § 7121(c)(1)</ref>, redesignated subsec. (d) as (e). Former subsec. (e) redesignated (f).</p>
<p style="-uslm-lc:I21" class="indent0">Subsec. (f). <ref href="/us/pl/108/458/s7121/c/1">Pub. L. 108–458, § 7121(c)(1)</ref>, redesignated subsec. (e) as (f).</p>
<p style="-uslm-lc:I21" class="indent0">1994—Subsec. (a). <ref href="/us/pl/103/437">Pub. L. 103–437</ref> substituted “Committee on Foreign Affairs” for “Committee on International Relations”.</p>
<p style="-uslm-lc:I21" class="indent0">1978—<ref href="/us/pl/95/426">Pub. L. 95–426</ref> designated existing provisions as subsec. (a), inserted “(including the text of any oral international agreement, which agreement shall be reduced to writing)”, and added subsecs. (b) to (e).</p>
<p style="-uslm-lc:I21" class="indent0">1977—<ref href="/us/pl/95/45">Pub. L. 95–45</ref> substituted “Committee on International Relations of the House of Representatives” for “Committee on Foreign Affairs of the House of Representatives” and inserted requirement that any department or agency of the United States Government which enters into any international agreement on behalf of the United States transmit to the Department of State the text of such agreement not later than twenty days after the agreement has been signed.</p>
</note>
<note style="-uslm-lc:I74" role="crossHeading" topic="statutoryNotes" id="idc8fb1a3b-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered"><b>Statutory Notes and Related Subsidiaries</b></heading></note>
<note style="-uslm-lc:I78" topic="changeOfName" id="idc8fb1a3c-d96f-11ed-bd9a-904c91f2bbbe">
<heading class="centered smallCaps">Change of Name</heading>
<p style="-uslm-lc:I21" class="indent0">Committee on International Relations of House of Representatives changed to Committee on Foreign Affairs of House of Representatives by House Resolution No. 6, One Hundred Tenth Congress, <date date="2007-01-05">Jan. 5, 2007</date>.</p>
</note>
<note style="-uslm-lc:I74" topic="effectiveDateOfAmendment" id="idc8fb1a3d-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Effective Date of 2022 Amendment</heading><p style="-uslm-lc:I21" class="indent0">Amendment by <ref href="/us/pl/117/263">Pub. L. 117–263</ref> effective 270 days after <date date="2022-12-23">Dec. 23, 2022</date>, see <ref href="/us/pl/117/263/s5947/c">section 5947(c) of Pub. L. 117–263</ref>, set out as a note under <ref href="/us/usc/t1/s112a">section 112a of this title</ref>.</p>
</note>
<note style="-uslm-lc:I74" topic="shortTitle" id="idc8fb1a3e-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Short Title</heading><p style="-uslm-lc:I21" class="indent0">This section is popularly known as the Case-Zablocki Act.</p>
</note>
<note style="-uslm-lc:I74" topic="miscellaneous" id="idc8fb1a3f-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Rules and Regulations</heading><p><ref href="/us/pl/117/263/dE/tLIX/s5947/a/5">Pub. L. 117–263, div. E, title LIX, § 5947(a)(5)</ref>, <date date="2022-12-23">Dec. 23, 2022</date>, <ref href="/us/stat/136/3482">136 Stat. 3482</ref>, provided that: <quotedContent origin="/us/pl/117/263/dE/tLIX/s5947/a/5">“Not later than 180 days after the date of the enactment of this Act [<date date="2022-12-23">Dec. 23, 2022</date>], the President, through the Secretary of State, shall promulgate such rules and regulations as may be necessary to carry out <ref href="/us/usc/t1/s112b">section 112b of title 1</ref>, United States Code, as amended by paragraph (1).”</quotedContent>
</p>
</note>
<note style="-uslm-lc:I74" topic="miscellaneous" id="idc8fb1a40-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Termination of Reporting Requirements</heading><p style="-uslm-lc:I21" class="indent0">For termination, effective <date date="2000-05-15">May 15, 2000</date>, of provisions of law requiring submittal to Congress of any annual, semiannual, or other regular periodic report listed in House Document No. 103–7 (in which the report required by subsec. (b) of this section is listed on page 38), see <ref href="/us/pl/104/66/s3003">section 3003 of Pub. L. 104–66</ref>, as amended, set out as a note under <ref href="/us/usc/t31/s1113">section 1113 of Title 31</ref>, Money and Finance.</p>
</note>
<note style="-uslm-lc:I74" topic="miscellaneous" id="idc8fb1a41-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Mechanism for Reporting</heading><p><ref href="/us/pl/117/263/dE/tLIX/s5947/a/4">Pub. L. 117–263, div. E, title LIX, § 5947(a)(4)</ref>, <date date="2022-12-23">Dec. 23, 2022</date>, <ref href="/us/stat/136/3481">136 Stat. 3481</ref>, provided that: <quotedContent origin="/us/pl/117/263/dE/tLIX/s5947/a/4">“Not later than 270 days after the date of the enactment of this Act [<date date="2022-12-23">Dec. 23, 2022</date>], the Secretary of State shall establish a mechanism for personnel of the Department of State who become aware or who have reason to believe that the requirements under <ref href="/us/usc/t1/s112b">section 112b of title 1</ref>, United States Code, as amended by paragraph (1), have not been fulfilled with respect to an international agreement or qualifying non-binding instrument (as such terms are defined in such section) to report such instances to the Secretary.”</quotedContent>
</p>
</note>
<note style="-uslm-lc:I74" topic="miscellaneous" id="idc8fb1a42-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Consultation and Briefing Requirement</heading><p><ref href="/us/pl/117/263/dE/tLIX/s5947/a/6">Pub. L. 117–263, div. E, title LIX, § 5947(a)(6)</ref>, <date date="2022-12-23">Dec. 23, 2022</date>, <ref href="/us/stat/136/3482">136 Stat. 3482</ref>, provided that:<quotedContent origin="/us/pl/117/263/dE/tLIX/s5947/a/6">
<subparagraph style="-uslm-lc:I21" class="indent0"><num value="A">“(A)</num><heading> <inline class="small-caps">Consultation</inline>.—</heading><content>The Secretary of State shall consult with the Committee on Foreign Relations of the Senate and the Committee on Foreign Affairs of the House of Representatives on matters related to the implementation of this section [amending this section, <ref href="/us/usc/t1/s112a">section 112a of this title</ref>, and <ref href="/us/usc/t6/s195c">section 195c of Title 6</ref>, Domestic Security, and enacting provisions set out as notes under this section and <ref href="/us/usc/t1/s112a">section 112a of this title</ref>] and the amendments made by this section before and after the effective date described in subsection (c) [see Effective Date of 2022 Amendment note set out under <ref href="/us/usc/t1/s112a">section 112a of this title</ref>].</content>
</subparagraph>
<subparagraph style="-uslm-lc:I21" class="indent0"><num value="B">“(B)</num><heading> <inline class="small-caps">Briefing</inline>.—</heading><content>Not later than 90 days after the date of the enactment of this Act [<date date="2022-12-23">Dec. 23, 2022</date>], and once every 90 days thereafter for 1 year, the Secretary shall brief the Committee on Foreign Relations of the Senate, the Committee on Appropriations of the Senate, the Committee on Foreign Affairs of the House of Representatives, and the Committee on Appropriations of the House of Representatives regarding the status of efforts to implement this section and the amendments made by this section.”</content>
</subparagraph>
</quotedContent>
</p>
</note>
<note style="-uslm-lc:I74" topic="miscellaneous" id="idc8fb1a43-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Enforcement</heading><p><ref href="/us/pl/100/204/tI/s139">Pub. L. 100–204, title I, § 139</ref>, <date date="1987-12-22">Dec. 22, 1987</date>, <ref href="/us/stat/101/1347">101 Stat. 1347</ref>, as amended by <ref href="/us/pl/108/458/tVII/s7121/e">Pub. L. 108–458, title VII, § 7121(e)</ref>, <date date="2004-12-17">Dec. 17, 2004</date>, <ref href="/us/stat/118/3808">118 Stat. 3808</ref>, provided that:<quotedContent origin="/us/pl/108/458/tVII/s7121/e">
<subsection style="-uslm-lc:I21" class="indent0"><num value="a">“(a)</num><heading> <inline class="small-caps">Restriction on Use of Funds</inline>.—</heading><content>If any international agreement, whose text is required to be transmitted to the Congress pursuant to the first sentence of subsection (a) of <ref href="/us/usc/t1/s112b">section 112b of title 1</ref>, United States Code (commonly referred to as the ‘Case-Zablocki Act’), is not so transmitted within the 60-day period specified in that sentence, then no funds authorized to be appropriated by this or any other Act shall be available after the end of that 60-day period to implement that agreement until the text of that agreement has been so transmitted.</content>
</subsection>
<subsection style="-uslm-lc:I21" class="indent0"><num value="b">“(b)</num><heading> <inline class="small-caps">Effective Date</inline>.—</heading><content>Subsection (a) shall take effect 60 days after the date of enactment of the 911 [probably means 9/11] Commission Implementation Act of 2004 [<date date="2004-12-17">Dec. 17, 2004</date>] and shall apply during fiscal years 2005, 2006, and 2007.”</content>
</subsection>
</quotedContent>
</p>
</note>
</notes>
</section>"""

In [3]:
# §113 - “Little and Brown’s” edition of laws and treaties; slip laws; Treaties and Other International Acts Series; admissibility in evidence
S113 = """<section style="-uslm-lc:I80" id="idc8fb1a44-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s113"><num value="113">§ 113.</num><heading> “Little and Brown’s” edition of laws and treaties; slip laws; Treaties and Other International Acts Series; admissibility in evidence</heading><content>
<p style="-uslm-lc:I11" class="indent0">The edition of the laws and treaties of the United States, published by Little and Brown, and the publications in slip or pamphlet form of the laws of the United States issued under the authority of the Archivist of the United States, and the Treaties and Other International Acts Series issued under the authority of the Secretary of State shall be competent evidence of the several public and private Acts of Congress, and of the treaties, international agreements other than treaties, and proclamations by the President of such treaties and international agreements other than treaties, as the case may be, therein contained, in all the courts of law and equity and of maritime jurisdiction, and in all the tribunals and public offices of the United States, and of the several States, without any further proof or authentication thereof.</p>
</content><sourceCredit id="idc8fb1a45-d96f-11ed-bd9a-904c91f2bbbe">(<ref href="/us/act/1947-07-30/ch388">July 30, 1947, ch. 388</ref>, <ref href="/us/stat/61/636">61 Stat. 636</ref>; <ref href="/us/pl/89/497/s1">Pub. L. 89–497, § 1</ref>, <date date="1966-07-08">July 8, 1966</date>, <ref href="/us/stat/80/271">80 Stat. 271</ref>; <ref href="/us/pl/98/497/tI/s107/d">Pub. L. 98–497, title I, § 107(d)</ref>, <date date="1984-10-19">Oct. 19, 1984</date>, <ref href="/us/stat/98/2291">98 Stat. 2291</ref>.)</sourceCredit>
<notes type="uscNote" id="idc8fb1a46-d96f-11ed-bd9a-904c91f2bbbe">
<note style="-uslm-lc:I74" role="crossHeading" topic="editorialNotes" id="idc8fb1a47-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered"><b>Editorial Notes</b></heading></note>
<note style="-uslm-lc:I74" topic="amendments" id="idc8fb1a48-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Amendments</heading><p style="-uslm-lc:I21" class="indent0">1984—<ref href="/us/pl/98/497">Pub. L. 98–497</ref> substituted “Archivist of the United States” for “Administrator of General Services”.</p>
<p style="-uslm-lc:I21" class="indent0">1966—<ref href="/us/pl/89/497">Pub. L. 89–497</ref> made slip laws and the Treaties and Other International Acts Series competent legal evidence of the several acts of Congress and the treaties and other international agreements contained therein.</p>
</note>
<note style="-uslm-lc:I74" role="crossHeading" topic="statutoryNotes" id="idc8fb1a49-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered"><b>Statutory Notes and Related Subsidiaries</b></heading></note>
<note style="-uslm-lc:I74" topic="effectiveDateOfAmendment" id="idc8fb1a4a-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Effective Date of 1984 Amendment</heading><p style="-uslm-lc:I21" class="indent0">Amendment by <ref href="/us/pl/98/497">Pub. L. 98–497</ref> effective <date date="1985-04-01">Apr. 1, 1985</date>, see <ref href="/us/pl/98/497/s301">section 301 of Pub. L. 98–497</ref>, set out as a note under <ref href="/us/usc/t44/s2102">section 2102 of Title 44</ref>, Public Printing and Documents.</p>
</note>
</notes>
</section>"""

In [4]:
# §114 - Sealing of instruments
S114 = """<section style="-uslm-lc:I80" id="idc8fb1a4b-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s114"><num value="114">§ 114.</num><heading> Sealing of instruments</heading><content>
<p style="-uslm-lc:I11" class="indent0">In all cases where a seal is necessary by law to any commission, process, or other instrument provided for by the laws of Congress, it shall be lawful to affix the proper seal by making an impression therewith directly on the paper to which such seal is necessary; which shall be as valid as if made on wax or other adhesive substance.</p>
</content><sourceCredit id="idc8fb1a4c-d96f-11ed-bd9a-904c91f2bbbe">(<ref href="/us/act/1947-07-30/ch388">July 30, 1947, ch. 388</ref>, <ref href="/us/stat/61/636">61 Stat. 636</ref>.)</sourceCredit>
</section>"""

In [5]:
# §201 - Publication and distribution of Code of Laws of United States and Supplements and District of Columbia Code and Supplements
S201 = """<section style="-uslm-lc:I80" id="idc8fb1a4f-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s201"><num value="201">§ 201.</num><heading> Publication and distribution of Code of Laws of United States and Supplements and District of Columbia Code and Supplements</heading>
<chapeau style="-uslm-lc:I11" class="indent0">In order to avoid duplication and waste—</chapeau><subsection style="-uslm-lc:I11" class="indent0" id="idc8fb1a50-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s201/a"><num value="a">(a)</num><heading> Publishing in slip or pamphlet form or in Statutes at Large.—</heading><content>Publication in slip or pamphlet form or in the Statutes at Large of any of the volumes or publications enumerated in sections 202 and 203 of this title, shall, in event of enactment, be dispensed with whenever the Committee on the Judiciary of the House of Representatives so directs the Archivist of the United States;</content>
</subsection>
<subsection style="-uslm-lc:I11" class="indent0" id="idc8fb1a51-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s201/b"><num value="b">(b)</num><heading> Curtailing number of copies published.—</heading><content>Curtailment of the number provided by law to be printed and distributed of the volumes or publications enumerated in sections 202 and 203 of this title may be directed by such committee, except that the Director of the Government Publishing Office shall print such numbers as are necessary for depository library distribution and for sale; and</content>
</subsection>
<subsection style="-uslm-lc:I11" class="indent0" id="idc8fb1a52-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s201/c"><num value="c">(c)</num><heading> Dispensing with publication of more than one Supplement for each Congress.—</heading><content>Such committee may direct that the printing and distribution of any supplement to the Code of Laws of the United States or to the Code of the District of Columbia be dispensed with entirely, except that there shall be printed and distributed for each Congress at least one supplement to each such code, containing the legislation of such Congress.</content>
</subsection>
<sourceCredit id="idc8fb1a53-d96f-11ed-bd9a-904c91f2bbbe">(<ref href="/us/act/1947-07-30/ch388">July 30, 1947, ch. 388</ref>, <ref href="/us/stat/61/637">61 Stat. 637</ref>; <ref href="/us/act/1954-09-03/ch1263/s1">Sept. 3, 1954, ch. 1263, § 1</ref>, <ref href="/us/stat/68/1226">68 Stat. 1226</ref>; <ref href="/us/pl/98/497/tI/s107/d">Pub. L. 98–497, title I, § 107(d)</ref>, <date date="1984-10-19">Oct. 19, 1984</date>, <ref href="/us/stat/98/2291">98 Stat. 2291</ref>; <ref href="/us/pl/113/235/dH/tI/s1301/d">Pub. L. 113–235, div. H, title I, § 1301(d)</ref>, <date date="2014-12-16">Dec. 16, 2014</date>, <ref href="/us/stat/128/2537">128 Stat. 2537</ref>.)</sourceCredit>
<notes type="uscNote" id="idc8fb1a54-d96f-11ed-bd9a-904c91f2bbbe">
<note style="-uslm-lc:I74" role="crossHeading" topic="editorialNotes" id="idc8fb1a55-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered"><b>Editorial Notes</b></heading></note>
<note style="-uslm-lc:I74" topic="amendments" id="idc8fb1a56-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Amendments</heading><p style="-uslm-lc:I21" class="indent0">1984—Subsec. (a). <ref href="/us/pl/98/497">Pub. L. 98–497</ref> substituted “Archivist of the United States” for “Administrator of General Services”.</p>
<p style="-uslm-lc:I21" class="indent0">1954—Subsec. (a). Act <date date="1954-09-03">Sept. 3, 1954</date>, substituted “Administrator of General Services” for “Secretary of State”.</p>
</note>
<note style="-uslm-lc:I74" role="crossHeading" topic="statutoryNotes" id="idc8fb1a57-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered"><b>Statutory Notes and Related Subsidiaries</b></heading></note>
<note style="-uslm-lc:I78" topic="changeOfName" id="idc8fb1a58-d96f-11ed-bd9a-904c91f2bbbe">
<heading class="centered smallCaps">Change of Name</heading>
<p style="-uslm-lc:I21" class="indent0">“Director of the Government Publishing Office” substituted for “Public Printer” in subsec. (b) on authority of <ref href="/us/pl/113/235/s1301/d">section 1301(d) of Pub. L. 113–235</ref>, set out as a note under <ref href="/us/usc/t44/s301">section 301 of Title 44</ref>, Public Printing and Documents.</p>
</note>
<note style="-uslm-lc:I74" topic="effectiveDateOfAmendment" id="idc8fb1a59-d96f-11ed-bd9a-904c91f2bbbe"><heading class="centered smallCaps">Effective Date of 1984 Amendment</heading><p style="-uslm-lc:I21" class="indent0">Amendment by <ref href="/us/pl/98/497">Pub. L. 98–497</ref> effective <date date="1985-04-01">Apr. 1, 1985</date>, see <ref href="/us/pl/98/497/s301">section 301 of Pub. L. 98–497</ref>, set out as a note under <ref href="/us/usc/t44/s2102">section 2102 of Title 44</ref>, Public Printing and Documents.</p>
</note>
</notes>
</section>"""

In [6]:
sections = [S112B, S113, S114, S201]
[embed.count_tokens(section) for section in sections]

[10971, 1083, 251, 1624]

In [7]:
try:
    embed.embed_one(S112B)
except openai.InvalidRequestError as error:
    print(error)

This model's maximum context length is 8191 tokens, however you requested 10971 tokens (10971 in your prompt; 0 for the completion). Please reduce your prompt; or completion length.


In [8]:
S112B_tree = ET.fromstring(S112B)

In [9]:
print(ET.tostring(S112B_tree, encoding='unicode'))

<section style="-uslm-lc:I80" id="idc8f8d033-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b"><num value="112b">§ 112b.</num><heading> United States international agreements; transmission to Congress</heading><subsection style="-uslm-lc:I11" class="indent0" id="idc8f8d034-d96f-11ed-bd9a-904c91f2bbbe" identifier="/us/usc/t1/s112b/a"><num value="a">(a)</num><content> The Secretary of State shall transmit to the Congress the text of any international agreement (including the text of any oral international agreement, which agreement shall be reduced to writing), other than a treaty, to which the United States is a party as soon as practicable after such agreement has entered into force with respect to the United States but in no event later than sixty days thereafter. However, any such agreement the immediate public disclosure of which would, in the opinion of the President, be prejudicial to the national security of the United States shall not be so transmitted to the Congress b

In [10]:
for S112B_element in S112B_tree.iter():
    S112B_element.attrib.clear()

In [11]:
print(ET.tostring(S112B_tree, encoding='unicode'))

<section><num>§ 112b.</num><heading> United States international agreements; transmission to Congress</heading><subsection><num>(a)</num><content> The Secretary of State shall transmit to the Congress the text of any international agreement (including the text of any oral international agreement, which agreement shall be reduced to writing), other than a treaty, to which the United States is a party as soon as practicable after such agreement has entered into force with respect to the United States but in no event later than sixty days thereafter. However, any such agreement the immediate public disclosure of which would, in the opinion of the President, be prejudicial to the national security of the United States shall not be so transmitted to the Congress but shall be transmitted to the Committee on Foreign Relations of the Senate and the Committee on International Relations of the House of Representatives under an appropriate injunction of secrecy to be removed only upon due notice 

In [12]:
def drop_attributes(element_text):
    tree = ET.fromstring(element_text)
    for element in tree.iter():
        element.attrib.clear()
    return ET.tostring(tree, encoding='unicode')

In [13]:
# Reminder of how many tokens the sections are *before* cleaning.
[embed.count_tokens(section) for section in sections]

[10971, 1083, 251, 1624]

In [14]:
cleaned_sections = [drop_attributes(section) for section in sections]
[embed.count_tokens(section) for section in cleaned_sections]

[7057, 591, 137, 852]

In [15]:
# That's much shorter. In THIS case, we can even use embed_many.
sum(_)

8637

In [16]:
embeddings = embed.embed_many(cleaned_sections)

In [17]:
# Let's try a query that non-semantic similarity is unlikely to help with. We
# embedded sections 112B, 113, 114, and 201. Section 114 answers this question.
embeddings @ embed.embed_one('Does embossing have official significance?')

array([0.7156805 , 0.73955655, 0.7810945 , 0.7316635 ], dtype=float32)

In [18]:
# Cool! But why did that work? If it worked for the most important of the
# reasons it should have, then that section should also have elevated
# similarity to single words related to embossing.
embeddings @ embed.embed_many(['emboss', 'embossed', 'embossing']).T

array([[0.6938791 , 0.69476515, 0.6941782 ],
       [0.70436895, 0.71381646, 0.70902956],
       [0.75279796, 0.7578374 , 0.7688054 ],
       [0.70862347, 0.71103317, 0.72231287]], dtype=float32)

In [19]:
# Let's search by an imperfect phrase. Should find 112B (the first one).
embeddings @ embed.embed_one('rules for how we keep track of diplomacy')

array([0.78910667, 0.7625172 , 0.7477622 , 0.7337751 ], dtype=float32)

In [20]:
# This should find 113 (the second one).
embeddings @ embed.embed_one('how to know if legislature wrote what my book says they did')

array([0.7321866 , 0.76482606, 0.7399651 , 0.7473894 ], dtype=float32)

In [21]:
# Can we use search with emojis? I'm not sure how well it works, but it makes
# sense that the globe emoji would be judged more similar to the first three,
# which mention treaties, than to the fourth, which doesn't.
embeddings @ embed.embed_one('🌐')

array([0.7103989 , 0.72968125, 0.72913504, 0.7074885 ], dtype=float32)

## Parsing experiments

In [51]:
P_ELEMENT = """<p>
  We have assumed <strong>all</strong> important text is located in a <em>leaf</em> of the tree.
</p>"""

In [52]:
p = ET.fromstring(P_ELEMENT)

In [53]:
p.text

'\n  We have assumed '

In [54]:
p.tail is None

True

In [55]:
p_children = list(p)
p_children

[<Element strong at 0x213e252c700>, <Element em at 0x213e252f380>]

In [56]:
strong, em = p_children

In [57]:
strong.text

'all'

In [58]:
strong.tail

' important text is located in a '

In [59]:
em.text

'leaf'

In [60]:
em.tail

' of the tree.\n'

In [ ]:
# TODO: Check if tail text is somehow preserved when an element is converted to a string.

In [49]:
def show_tails(root):
    for _, element in ET.iterwalk(root, events=('start',)):
        if element.tail is None:
            continue
        tail = element.tail.strip()
        if tail:
            print(f'{element}: {element.tail!r}')

In [50]:
show_tails(ET.fromstring(S112B))

<Element date at 0x213e45ead00>: ', and at yearly intervals thereafter, the President shall, under his own signature, transmit to the Speaker of the House of Representatives and the chairman of the Committee on Foreign Relations of the Senate a report with respect to each international agreement which, during the preceding year, was transmitted to the Congress after the expiration of the 60-day period referred to in the first sentence of subsection (a), describing fully and completely the reasons for the late transmittal.'
<Element note at 0x213e45f7180>: ' of the Export Administration Act of 1979 (50 U.S.C. App. 2405(j)(1)(A)), section 620A(a) of the Foreign Assistance Act of 1961 ('
<Element num at 0x213e45f4d40>: '\u202fSee References in Text note below.'
<Element ref at 0x213e45f6dc0>: '), or section 40(d) of the Arms Export Control Act ('
<Element ref at 0x213e45f4e00>: ').'
<Element ref at 0x213e45f4e00>: ', '
<Element date at 0x213e45f5540>: ', '
<Element ref at 0x213e45f4d40>: 